In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import collections
import os
import glob
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go
from typing import List, Dict, Tuple



In [3]:
## TODO: read the interface/device from couples.csv daliy and report the totally missing interface/device {DONE}
## TODO: read the multiple days files {DONE}
## TODO: use case {DONE}
# use case 1: totally missing interface/device {DONE}
# use case 2: partially missing interface/device (be recovered) {DONE}
# use case 3: missing interface/device without recovery (lasting missing values until previous midnight){DONE}

In [4]:
## Read the all the folders in the folder apcn/cloud
## The name of the folders is the date of the day
## Read data from each folder: DAILY_IN.csv, INPUT.csv, OUTPUT.csv, DAILY_OUT.csv, couples.csv
## Read each csv file and do not set the first row as header
## For each day, return 5 dataframes: df_dayin, df_input, df_dayout, df_output, df_couples

def read_data(path):
    df_dayin = pd.read_csv(path + '/DAILY_IN.csv', sep=';', header = None)
    df_input = pd.read_csv(path + '/INPUT.csv', sep=';', header = None)
    df_dayout = pd.read_csv(path + '/DAILY_OUT.csv', sep=';', header = None)
    df_output = pd.read_csv(path + '/OUTPUT.csv', sep=';', header = None)
    df_couples = pd.read_csv(path + '/couples.csv', sep=';', header=None)
    return df_dayin, df_input, df_dayout, df_output, df_couples

## Read the dataframe again and add the header as the column names
def set_header(df, header):
    df.columns = header
    return df


In [5]:
df_dayin, df_input, df_dayout, df_output, df_couples = read_data('data/apcn/2023-03-27')

In [6]:
## Set the header for each dataframe

set_header(df_couples, ['id', 'router', 'subinterface', 'SAMPLES', 'infrastructure'])

# TODO:get the accurate header {DONE}
# TODO: keep the relative coulumn and drop the others {DONE}
set_header(df_dayin, ['id', 'router', 'INTERFACE', 'DATE', 'MAXvalue', 'AVERAGE', 'PERCENTILE', 'CROSS', 'RATIOMAX', 
                      'RATIOAVERAGE', 'RATIOPERCENTILE', 'SAMPLES', 'infrastrution'])


# TODO:get the accurate header {DONE}
# TODO: keep the relative coulumn and drop the others {DONE}
# set_header(df_dayout, ['id', 'router', 'INTERFACE', 'DATE', 'MAXvalue', 'AVERAGE', 'PERCENTILE', 'CROSS', 'RATIOMAX', 
#                       'RATIOAVERAGE', 'RATIOPERCENTILE', 'SAMPLES', 'infrastrution'])


set_header(df_input, ['id', 'router', 'subinterface', 'timestamp', 'value'])

# set_header(df_out, ['id', 'router', 'subinterface', 'timestamp', 'value'])

,id,router,subinterface,timestamp,value
0,2319479,PSEL905,Serial0/0/0.101,2023-03-26 00:00:21,0.287200
1,2319479,PSEL905,Serial0/0/0.101,2023-03-26 00:05:21,0.289626
2,2319479,PSEL905,Serial0/0/0.101,2023-03-26 00:10:21,0.284500
3,2319479,PSEL905,Serial0/0/0.101,2023-03-26 00:15:21,0.270600
4,2319479,PSEL905,Serial0/0/0.101,2023-03-26 00:20:21,0.280934
...,...,...,...,...,...
221249,2314832,PABV174,Tunnel1114,2023-03-26 23:35:01,0.039037
221250,2314832,PABV174,Tunnel1114,2023-03-26 23:40:01,0.044100
221251,2314832,PABV174,Tunnel1114,2023-03-26 23:45:01,0.041162
221252,2314832,PABV174,Tunnel1114,2023-03-26 23:50:01,0.041967


In [7]:
def get_subinterfaces_less_than_d_times(df: pd.DataFrame, d: int, n: int) -> list:
    """
    This function takes a DataFrame of timestamps and the number of days (d) and the number of missing timestamps per day (n).
    It returns a list of tuples, each containing the subinterface number, router name, and the count of timestamps for that subinterface,
    for all subinterfaces that have a count less than d*(288-n).
    """
    # Group the DataFrame by router and subinterface, and count the number of timestamps for each subinterface
    timestamps_count = df.groupby(['router', 'subinterface'])['timestamp'].count()
    
    # Calculate the minimum number of timestamps required for a subinterface
    min_timestamps = d * (288 - n)
    
    # Filter subinterfaces that have a count less than the minimum required
    subinterfaces_less_than_d_times = timestamps_count[timestamps_count < min_timestamps]
    
    # Retrieve the router name, subinterface, and count for each subinterface
    subinterfaces_info = [(router, subinterface, count) for (router, subinterface), count in subinterfaces_less_than_d_times.items()]
    
    return subinterfaces_info


In [8]:
d = 1 # numbers of files instead of days
n = 30

subinterfaces_info = get_subinterfaces_less_than_d_times(df_input, d, n)

# Iterate over the subinterfaces and print the information
for router, subinterface, count in subinterfaces_info:
    min_timestamps = d * (288 - n)
    print(f"The timestamps of (interface: {subinterface} ROUTER: {router}) less than {min_timestamps}, which number is {count}.")
else:
    print(f"No subinterface has less than {d*(288-n)} timestamps.")


The timestamps of (interface: Tunnel1113 ROUTER: PAJA255) less than 258, which number is 126.
The timestamps of (interface: Tunnel0 ROUTER: PAQJ006) less than 258, which number is 174.
The timestamps of (interface: FastEthernet0/1 ROUTER: PASM025) less than 258, which number is 171.
The timestamps of (interface: Serial0/0/0.105 ROUTER: PASM037) less than 258, which number is 130.
The timestamps of (interface: Tunnel1116 ROUTER: PASM037) less than 258, which number is 129.
The timestamps of (interface: Tunnel1113 ROUTER: PBDS093) less than 258, which number is 257.
The timestamps of (interface: Serial0/0/0:0.102 ROUTER: PCGP079) less than 258, which number is 51.
The timestamps of (interface: GigabitEthernet0/1.11 ROUTER: PCNS051) less than 258, which number is 27.
The timestamps of (interface: Tunnel0 ROUTER: PEBL011) less than 258, which number is 217.
The timestamps of (interface: Tunnel1113 ROUTER: PEIN655) less than 258, which number is 239.
The timestamps of (interface: Tunnel1114

In [9]:
def visualize_subinterfaces(df: pd.DataFrame, subinterfaces_info: list):
    """
    This function visualizes the subinterfaces by plotting line charts for each subinterface.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamps and values.
        subinterfaces_info (list): List of tuples containing the subinterface information.

    """
    
    # Create an empty figure
    fig = go.Figure()

    # Iterate over the subinterfaces
    for router, subinterface, count in subinterfaces_info:
        # Filter the DataFrame for the current subinterface
        sub_df = df[(df['router'] == router) & (df['subinterface'] == subinterface)].copy()

        # Convert the timestamp column to datetime type
        sub_df['timestamp'] = pd.to_datetime(sub_df['timestamp'])

        # Create a DataFrame with all possible timestamps for the subinterface
        timestamps = pd.date_range(start=sub_df['timestamp'].min(), end=sub_df['timestamp'].max(), freq='5min')
        fill_df = pd.DataFrame({'timestamp': timestamps})

        # Convert the timestamp column to datetime type
        fill_df['timestamp'] = pd.to_datetime(fill_df['timestamp'])

        # Merge the original DataFrame with the DataFrame of all timestamps
        merged_df = pd.merge(fill_df, sub_df, on='timestamp', how='left')

        # Add a line trace to the figure
        fig.add_trace(
            go.Scatter(
                x=merged_df['timestamp'],
                y=merged_df['value'],
                mode='lines',
                name=f"Subinterface {subinterface} (Router {router})"
            )
        )

    # Set the x-axis label and title
    fig.update_xaxes(title_text='Timestamp')
    fig.update_layout(title_text='Missing Interface Visualization')

    # Show the plot
    fig.show()


In [10]:
visualize_subinterfaces(df_input, subinterfaces_info)


In [11]:
# Detect missing interfaces without recovery
def detect_missing_interfaces_without_recovery(df: pd.DataFrame, min_timestamps: int) -> list:
    missing_interfaces = []
    for (router, subinterface), group_df in df.groupby(['router', 'subinterface']):
        # Convert timestamp column to datetime
        group_df['timestamp'] = pd.to_datetime(group_df['timestamp'])
        
        # Check if there are missing timestamps until midnight
        if len(group_df) < min_timestamps and group_df['timestamp'].dt.hour.max() < 23:
            missing_interfaces.append((router, subinterface))
    return missing_interfaces

# Detect partially missing interfaces
def detect_partially_missing_interfaces(df: pd.DataFrame, min_timestamps: int, threshold: int) -> list:
    partially_missing_interfaces = []
    for (router, subinterface), group_df in df.groupby(['router', 'subinterface']):
        # Convert timestamp column to datetime
        group_df['timestamp'] = pd.to_datetime(group_df['timestamp'])
        
        # Check if there are partially missing timestamps
        if len(group_df) < min_timestamps and group_df['timestamp'].dt.hour.max() >= 23:
            missing_timestamps_after_23 = group_df[group_df['timestamp'].dt.hour >= 23]
            if len(missing_timestamps_after_23) >= threshold:
                partially_missing_interfaces.append((router, subinterface))
    return partially_missing_interfaces

In [12]:
def visualize_missing_interfaces(df: pd.DataFrame, missing_interfaces: list, title: str):
    """
    This function visualizes the missing interfaces by creating line charts for each subinterface.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamps and values.
        missing_interfaces (list): List of tuples containing the missing interface information.
        title (str): The title of the line chart.

    """
    # Create an empty figure
    fig = go.Figure()

    # Iterate over the missing interfaces
    for router, subinterface in missing_interfaces:
        # Filter the DataFrame for the current subinterface
        sub_df = df[(df['router'] == router) & (df['subinterface'] == subinterface)].copy()

        # Convert the timestamp column to datetime type
        sub_df['timestamp'] = pd.to_datetime(sub_df['timestamp'])

        # Add a line trace to the figure
        fig.add_trace(
            go.Scatter(
                x=sub_df['timestamp'],
                y=sub_df['value'],
                mode='lines',
                name=f"Subinterface {subinterface} (Router {router})"
            )
        )

    # Set the x-axis label and title
    fig.update_xaxes(title_text='Timestamp')
    fig.update_layout(title_text=title)

    # Show the plot
    fig.show()


In [18]:
# Define the minimum number of timestamps and recovery threshold
min_timestamps = 258  # Minimum number of timestamps for a complete day (288 for 5-minute intervals)
threshold = 12  # Recovery threshold (number of timestamps from 23:00 to end of day)

# Detect missing interfaces without recovery
missing_interfaces = detect_missing_interfaces_without_recovery(df_input, min_timestamps)
if len(missing_interfaces) > 0:
    for router, subinterface in missing_interfaces:
        print(f"The timestamps of (interface: {subinterface} ROUTER: {router}) without recovery.")
else:
    print("No interfaces without recovery found.")

# Detect partially missing interfaces
partially_missing_interfaces = detect_partially_missing_interfaces(df_input, min_timestamps, threshold)
if len(partially_missing_interfaces) > 0:
    for router, subinterface in partially_missing_interfaces:
        print(f"The timestamps of (interface: {subinterface} ROUTER: {router}) is partially missing and recovered.")
else:
    print("No partially missing interfaces found.")

The timestamps of (interface: Tunnel1113 ROUTER: PAJA255) without recovery.
The timestamps of (interface: Tunnel0 ROUTER: PAQJ006) without recovery.
The timestamps of (interface: Tunnel1116 ROUTER: PASM037) without recovery.
The timestamps of (interface: GigabitEthernet0/1.11 ROUTER: PCNS051) without recovery.
The timestamps of (interface: GigabitEthernet0/2.339 ROUTER: PHKT048) without recovery.
The timestamps of (interface: Tunnel0 ROUTER: PMEX2238) without recovery.
The timestamps of (interface: FastEthernet0/1 ROUTER: PASM025) is partially missing and recovered.
The timestamps of (interface: Serial0/0/0.105 ROUTER: PASM037) is partially missing and recovered.
The timestamps of (interface: Tunnel1113 ROUTER: PBDS093) is partially missing and recovered.
The timestamps of (interface: Serial0/0/0:0.102 ROUTER: PCGP079) is partially missing and recovered.
The timestamps of (interface: Tunnel1113 ROUTER: PEIN655) is partially missing and recovered.
The timestamps of (interface: Tunnel111

In [19]:
# Visualize missing interfaces without recovery
visualize_missing_interfaces(df_input, missing_interfaces, "Missing interfaces without recovery")

# Visualize partially missing interfaces
visualize_missing_interfaces(df_input, partially_missing_interfaces, "Partially missing interfaces with recovery")


In [20]:
## read the interface/device from df_couples and df_input and report the unmatched interface/device
## the unmatched interface/device: the interface/device in df_couples but not in df_input

def get_unmatched_interfaces(df_couples: pd.DataFrame, df_input: pd.DataFrame):
    """
    This function reads the ('subinterface' and 'router') from df_couples and df_input
    and reports the unmatched ('subinterface' and 'router').

    Args:
        df_couples (pd.DataFrame): DataFrame containing the ('subinterface' and 'router') from df_couples.
        df_input (pd.DataFrame): DataFrame containing the ('subinterface' and 'router') from df_input.

    """
    # Get the unique ('subinterface' and 'router') pairs from df_couples
    couples = set(zip(df_couples['subinterface'], df_couples['router']))

    # Get the unique ('subinterface' and 'router') pairs from df_input
    inputs = set(zip(df_input['subinterface'], df_input['router']))

    # Find the unmatched ('subinterface' and 'router') pairs
    unmatched = couples - inputs

    # Print the unmatched ('subinterface' and 'router') pairs
    print("The totally missing (interface, router) are:")
    for subinterface, router in unmatched:
        print(f"{router} ({subinterface})")


In [21]:
# Call the get_unmatched_interfaces function
get_unmatched_interfaces(df_couples, df_input)
## TODO: add the table to show

The totally missing (interface, router) are:
PSEA081 GigabitEthernet0/0/2.60)
PNAP225 GigabitEthernet0/1.485)
PDNK070 Tunnel1114)
PTNR045 FastEthernet0/1.28)
PBOM817 GigabitEthernet0/1.3)
PGRK019 ATM0.1-aal5 layer)
PKWI403 Tunnel0)
PNTE564 Tunnel1116)
PKRR067 GigabitEthernet0/1.713)
POAX008 Tunnel0)
PAQJ009 Tunnel1114)
PSEL1302 Serial0/0/0.101)
PBJV003 Serial0/0/0.101)
PLON9363 GigabitEthernet0/1.379)
PSHO001 Serial0/0/0.101)
PCAI918 GigabitEthernet0/2.1805)
PJER051 Tunnel1114)
PPUS236 Tunnel1122)
PFLR261 GigabitEthernet0/1.118)
PORD199 GigabitEthernet0/0/0.234)
PTLS1007 Tunnel1114)
PVVO056 FastEthernet0/1.21)
PBEY371 MFR1.102)
PXRE214 GigabitEthernet0/1.154)
PABJ363 GigabitEthernet0/0/1.11)
PPVG076 GigabitEthernet0/1.121)
PFRA1021 GigabitEthernet0/0/1.711)
PLGA103 Tunnel0)
PTUN299 GigabitEthernet0/1.5)
PMXL043 Tunnel0)
PLVI006 ATM0.1-aal5 layer)
PDOH211 GigabitEthernet0/2.411)
PCAI917 FastEthernet0/1/1.1616)
PYYC337 ATM0.1-aal5 layer)
PZLO027 GigabitEthernet0/0/1.409)
PRIC201 Tunnel0)

In [ ]:
#TODO: create a single dashboard to show all the charts and one table
#TODO: sort by the router name and subinterface for all the charts and table
#TODO: find a way to read the csv files into the dashboard
#TODO: maybe keep the last 7 days data

